In [1]:
!pip install tensorflow tensorflow-hub librosa pandas openai-whisper ffmpeg-python
!pip install torch librosa git+https://github.com/openai/whisper.git pyannote.audio transformers langdetect
!pip install langdetect pyannote.audio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 17.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=1a95055fcbc30eec890789ee2f375d075ff027e8f1d8398a4886a82f13e7ea6c
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-nufskru5
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-nufskru5
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import torch
import librosa
import whisper
from pyannote.audio import Pipeline
from transformers import pipeline as hf_pipeline
import subprocess, json
from pathlib import Path

# ===================== CONFIG =====================
AUDIO_FILE = "/content/audio_001.mp3" 
OUTPUT_JSON = "final_output.json"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# ===================== HELPER =====================
def convert_to_wav(input_path, output_path="temp.wav"):
    """Convert mp4/mkv etc. to wav for processing"""
    subprocess.run([
        "ffmpeg", "-y", "-i", input_path,
        "-ar", "16000", "-ac", "1", output_path
    ], check=True)
    return output_path

# ===================== 1. SOUND EVENT DETECTION =====================
def run_sed(audio_path, threshold=0.01):
    sed_pipeline = hf_pipeline(
        task="audio-classification",
        model="MIT/ast-finetuned-audioset-10-10-0.4593",
        device=0 if DEVICE == "cuda" else -1
    )
    audio_data, sr = librosa.load(audio_path, sr=16000)
    results = sed_pipeline(audio_data)
    # keep only non-speech sounds above threshold
    results = [
        e for e in results
        if "speech" not in e["label"].lower() and e["score"] >= threshold
    ]
    return results

# ===================== 2. SPEAKER DIARIZATION =====================
def run_diarization(audio_path):
    pipeline = Pipeline.from_pretrained(
        "pyannote/speaker-diarization@2.1",
        use_auth_token="YOUR_TOKEN" 
    )
    diarization = pipeline(audio_path)
    segments = []
    for turn, _, speaker in diarization.itertracks(yield_label=True):
        segments.append({
            "start": float(turn.start),
            "end": float(turn.end),
            "speaker": speaker
        })
    return segments

# ===================== 3. SPEECH TO TEXT =====================
def run_stt(audio_path):
    model = whisper.load_model("medium", device=DEVICE)
    transcription = model.transcribe(audio_path)
    detected_lang = transcription["language"]
    segments = []
    for seg in transcription["segments"]:
        segments.append({
            "start_time": seg["start"],
            "end_time": seg["end"],
            "text": seg["text"],
            "confidence": seg.get("avg_logprob", None)
        })
    return detected_lang, segments

# ===================== 4. EMOTION DETECTION =====================
def run_emotion_detection(texts):
    emo_pipeline = hf_pipeline(
        "text-classification",
        model="j-hartmann/emotion-english-distilroberta-base",
        top_k=None,
        device=0 if DEVICE == "cuda" else -1
    )
    results = emo_pipeline(texts)
    # keep only top emotion
    emotions = []
    for res in results:
        if isinstance(res, list):
            res = sorted(res, key=lambda x: x["score"], reverse=True)[0]
        emotions.append({"label": res["label"], "score": float(res["score"])})
    return emotions

# ===================== 5. MERGE & SAVE JSON =====================
def build_output(file_id, audio_path, sed_events, diarization, transcript, detected_lang):
    # ---- attach speakers to transcript ----
    for t in transcript:
        t["speaker"] = "Unknown"
        for s in diarization:
            if (s["start"] <= t["start_time"] <= s["end"]) or (s["start"] <= t["end_time"] <= s["end"]):
                t["speaker"] = s["speaker"]
                break

    # ---- add emotion detection ----
    texts = [t["text"] for t in transcript]
    emotions = run_emotion_detection(texts)
    for t, emo in zip(transcript, emotions):
        t["emotion"] = emo["label"]
        t["emotion_score"] = emo["score"]

    merged_transcript = [
        {
            "speaker": t["speaker"],
            "start_time": t["start_time"],
            "end_time": t["end_time"],
            "text": t["text"].strip(),
            "emotion": t["emotion"],
            "emotion_score": round(t["emotion_score"], 3)
        }
        for t in transcript
    ]

    # ---- sort and clean sound effects ----
    sorted_sounds = sorted(sed_events, key=lambda x: x["score"], reverse=True)
    sound_effects = [
        {"label": s["label"], "score": round(s["score"], 3)}
        for s in sorted_sounds
    ]

    # ---- final JSON ----
    data = {
        "file_id": Path(file_id).stem,
        "file_path": audio_path,
        "detected_language": detected_lang,
        "transcript": merged_transcript,
        "sound_effects": sound_effects,
        "metadata": {
            "annotator": "system_auto"
        }
    }

    with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)
    print(f"\n✅ Final JSON saved to {OUTPUT_JSON}")

# ===================== MAIN =====================
if __name__ == "__main__":
    wav_path = convert_to_wav(AUDIO_FILE)

    print("Running SED...")
    sed_events = run_sed(wav_path)

    print("Running diarization...")
    diarization = run_diarization(wav_path)

    print("Running STT...")
    detected_lang, transcript = run_stt(wav_path)

    print("Building JSON with emotions and timestamps...")
    build_output(AUDIO_FILE, AUDIO_FILE, sed_events, diarization, transcript, detected_lang)


Running SED...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0



✅ Final JSON saved to final_output.json
